In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
import sklearn
import nltk.collocations 
from nltk import FreqDist, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string, re
import urllib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cmiw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cmiw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cmiw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\cmiw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
import pandas

lyrics = pandas.read_csv('lyrics-data.csv')
english_songs = lyrics['Idiom'] =='ENGLISH'

In [3]:
english = lyrics[english_songs]

In [4]:
artist = pandas.read_csv('artists-data.csv')
print(artist.shape)
pop = artist['Genre'] == 'Pop'
rock = artist['Genre'] == 'Rock'
hiphop = artist['Genre'] == 'Hip Hop'

popsongs = artist[pop]
rocksongs = artist[rock]
hiphopsongs = artist[hiphop]

english_popsongs = pandas.merge(popsongs, english, left_on='Link', right_on='ALink')
english_popsongs.to_csv('englishPopSongs.csv', index=False)

english_rocksongs = pandas.merge(rocksongs, english, left_on='Link', right_on='ALink')
english_rocksongs.to_csv('englishRockSongs.csv', index=False)

english_hiphopsongs = pandas.merge(hiphopsongs, english, left_on='Link', right_on='ALink')
english_hiphopsongs.to_csv('englishHipHopSongs.csv', index=False)

(3242, 6)


In [5]:
import pandas as pd

frames = [english_popsongs, english_rocksongs, english_hiphopsongs] 
english_allsong = pd.concat(frames)
english_allsong

,Artist,Songs,Popularity,Link,Genre,Genres,ALink,SName,SLink,Lyric,Idiom
0,Adam Lambert,110,1.4,/adam-lambert/,Pop,Pop; Pop/Rock; Rock; Romântico; Dance; Electro...,/adam-lambert/,Whataya Want From Me,/adam-lambert/whataya-want-from-me.html,"Hey, slow it down. What do you want from me. W...",ENGLISH
1,Adam Lambert,110,1.4,/adam-lambert/,Pop,Pop; Pop/Rock; Rock; Romântico; Dance; Electro...,/adam-lambert/,Ghost Town,/adam-lambert/ghost-town.html,Died last night in my dreams. Walking the stre...,ENGLISH
2,Adam Lambert,110,1.4,/adam-lambert/,Pop,Pop; Pop/Rock; Rock; Romântico; Dance; Electro...,/adam-lambert/,We Are The Champions (feat. Kris Allen & Queen),/adam-lambert/we-are-the-champions-feat-kris-a...,I've paid my dues. Time after time. I've done ...,ENGLISH
3,Adam Lambert,110,1.4,/adam-lambert/,Pop,Pop; Pop/Rock; Rock; Romântico; Dance; Electro...,/adam-lambert/,If I Had You,/adam-lambert/if-i-had-you.html,"So I got my boots on,. Got the right amount of...",ENGLISH
4,Adam Lambert,110,1.4,/adam-lambert/,Pop,Pop; Pop/Rock; Rock; Romântico; Dance; Electro...,/adam-lambert/,Never Close Our Eyes,/adam-lambert/never-close-our-eyes.html,I wish that this night would never be over. Th...,ENGLISH
...,...,...,...,...,...,...,...,...,...,...,...
23103,US5,42,0.1,/us5/,Hip Hop,Pop; Dance; Hip Hop; Romântico; World Music; R&B,/us5/,Work Your Body,/us5/work-your-body.html,Work your body. I thought I told ya that we wo...,ENGLISH
23104,US5,42,0.1,/us5/,Hip Hop,Pop; Dance; Hip Hop; Romântico; World Music; R&B,/us5/,Your Love,/us5/your-love.html,"Verse 1. Girl, the only way to make things rig...",ENGLISH
23105,Furacão 2000,57,2.8,/furacao-2000/,Hip Hop,Funk Carioca; Axé; Hip Hop; Rap; Dance; Funk; ...,/furacao-2000/,How Can We Be Wrong (melô do Anjinho),/furacao-2000/how-can-we-be-wrong-melo-do-anji...,How long can we both go on this way. Why can't...,ENGLISH
23106,Pancadão do Caldeirão do Huck,30,0.2,/pancadao-do-caldeirao-do-huck/,Hip Hop,Funk Carioca; Electronica; Hip Hop; Black Musi...,/pancadao-do-caldeirao-do-huck/,Chanson Du Soleil,/pancadao-do-caldeirao-do-huck/chanson-du-sole...,Back in the day. I wasn't sure. You were afrai...,ENGLISH


In [6]:
english_allsong.dropna(inplace = True)
english_allsong.isna().sum()
english_allsong.reset_index(inplace = True)

In [7]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?_\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [8]:
data_clean = pandas.DataFrame(english_allsong['Lyric'].apply(round1))
Genre_list = english_allsong['Genre']
data_clean['Genre'] = Genre_list
data_clean

,Lyric,Genre
0,hey slow it down what do you want from me what...,Pop
1,died last night in my dreams walking the stree...,Pop
2,ive paid my dues time after time ive done my s...,Pop
3,so i got my boots on got the right amount of l...,Pop
4,i wish that this night would never be over the...,Pop
...,...,...
123981,work your body i thought i told ya that we won...,Hip Hop
123982,verse girl the only way to make things right ...,Hip Hop
123983,how long can we both go on this way why cant y...,Hip Hop
123984,back in the day i wasnt sure you were afraid w...,Hip Hop


In [9]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [10]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.Lyric.apply(round2))
Genre_list = english_allsong['Genre']
data_clean['Genre'] = Genre_list
data_clean

,Lyric,Genre
0,hey slow it down what do you want from me what...,Pop
1,died last night in my dreams walking the stree...,Pop
2,ive paid my dues time after time ive done my s...,Pop
3,so i got my boots on got the right amount of l...,Pop
4,i wish that this night would never be over the...,Pop
...,...,...
123981,work your body i thought i told ya that we won...,Hip Hop
123982,verse girl the only way to make things right ...,Hip Hop
123983,how long can we both go on this way why cant y...,Hip Hop
123984,back in the day i wasnt sure you were afraid w...,Hip Hop


In [11]:
data_clean.Genre.value_counts()

Rock       60584
Pop        40294
Hip Hop    23108
Name: Genre, dtype: int64

In [12]:
temp_data_df = data_clean
temp_data_df

,Lyric,Genre
0,hey slow it down what do you want from me what...,Pop
1,died last night in my dreams walking the stree...,Pop
2,ive paid my dues time after time ive done my s...,Pop
3,so i got my boots on got the right amount of l...,Pop
4,i wish that this night would never be over the...,Pop
...,...,...
123981,work your body i thought i told ya that we won...,Hip Hop
123982,verse girl the only way to make things right ...,Hip Hop
123983,how long can we both go on this way why cant y...,Hip Hop
123984,back in the day i wasnt sure you were afraid w...,Hip Hop


In [13]:
rock_df = data_clean[data_clean.Genre == "Rock"]
pop_df = data_clean[data_clean.Genre == "Pop"]
hip_df = data_clean[data_clean.Genre == "Hip Hop"]

In [14]:
data_clean.drop(data_clean[data_clean.Genre == "Rock"].index, inplace = True)
data_clean.drop(data_clean[data_clean.Genre == "Pop"].index, inplace = True)
data_clean.drop(data_clean[data_clean.Genre == "Hip Hop"].index, inplace = True)

In [15]:
maybe_df = data_clean.append([rock_df, pop_df, hip_df])
maybe_df.to_pickle("corpus.pkl")
maybe_df.Genre.value_counts()

Rock       60584
Pop        40294
Hip Hop    23108
Name: Genre, dtype: int64

In [16]:
maybe_df.reset_index(inplace = True)
maybe_df

,index,Lyric,Genre
0,40294,i could feel at the time there was no way of k...,Rock
1,40295,take me now baby here as i am hold me close an...,Rock
2,40296,these are these are days youll remember never ...,Rock
3,40297,a lie to say o my mountain has coal veins and ...,Rock
4,40298,trudging slowly over wet sand back to the benc...,Rock
...,...,...,...
123981,123981,work your body i thought i told ya that we won...,Hip Hop
123982,123982,verse girl the only way to make things right ...,Hip Hop
123983,123983,how long can we both go on this way why cant y...,Hip Hop
123984,123984,back in the day i wasnt sure you were afraid w...,Hip Hop


In [17]:
import re
import string

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
english = list(set(nltk.corpus.words.words()))

def clean_docs_lemma(text):
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    for lyric in text:
        clean_lyric = nltk.regexp_tokenize(lyric, pattern)
        lyric_lower = [i.lower() for i in clean_lyric]
        stop_words = stopwords.words('english')
        stopwords_list = stop_words + list(string.punctuation)
        stopwords_list += ["''", '""', '...', '``']
        lyrics_tokens_stopped = [w for w in lyric_lower if not w in stopwords_list]
        lyric_lemmas = [lemmatizer.lemmatize(word) for word in lyrics_tokens_stopped]
        c = " ".join(lyric_lemmas)
        text = text + c
    return text
lemmatized = lambda x: clean_docs_lemma(x)

In [ ]:
data_clean_lemma = pd.DataFrame(maybe_df.Lyric.apply(lemmatized))
Genre_list = maybe_df['Genre']
data_clean_lemma['Genre'] = Genre_list
data_clean_lemma

In [ ]:
import re
import string

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
english = list(set(nltk.corpus.words.words()))

def clean_docs_stemma(text):
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    for lyric in text:
        clean_lyric = nltk.regexp_tokenize(lyric, pattern)
        lyric_lower = [i.lower() for i in clean_lyric]
        stop_words = stopwords.words('english')
        stopwords_list = stop_words + list(string.punctuation)
        stopwords_list += ["''", '""', '...', '``']
        lyrics_tokens_stopped = [w for w in lyric_lower if not w in stopwords_list]
        lyric_stemmas = [stemmer.stem(word) for word in lyrics_tokens_stopped]
        c = " ".join(lyric_stemmas)
        text = text + c
    return text

stemmatized = lambda x: clean_docs_stemma(x)

In [ ]:
data_clean_stemma = pd.DataFrame(maybe_df.Lyric.apply(stemmatized))
Genre_list = maybe_df['Genre']
data_clean_stemma['Genre'] = Genre_list
data_clean_stemma

In [ ]:
data_clean_lemma.to_pickle("corpus.pkl")

In [ ]:
rock_df = pd.DataFrame(data_clean_lemma)
pop_df = pd.DataFrame(data_clean_lemma)
hiphop_df = pd.DataFrame(data_clean_lemma)

In [ ]:
rock_df

In [ ]:
cv_rock = CountVectorizer(stop_words='english')
data_cv = cv_rock.fit_transform(rock_df.Lyric)
data_dtm_rock = pd.DataFrame(data_cv.toarray(), columns=cv_rock.get_feature_names())
data_dtm_rock.index = rock_df.index
data_dtm_rock

In [ ]:
cv_pop = CountVectorizer(stop_words='english')
data_cv = cv_pop.fit_transform(pop_df.Lyric)
data_dtm_pop = pd.DataFrame(data_cv.toarray(), columns=cv_pop.get_feature_names())
data_dtm_pop.index = pop_df.index
data_dtm_pop

In [ ]:
cv_hiphop = CountVectorizer(stop_words='english')
data_cv = cv_hiphop.fit_transform(hiphop_df.Lyric)
data_dtm_hiphop = pd.DataFrame(data_cv.toarray(), columns=cv_hiphop.get_feature_names())
data_dtm_hiphop.index = hiphop_df.index
data_dtm_hiphop

In [ ]:
data_dtm_rock.to_pickle("dtm_rock.pkl")
data_dtm_pop.to_pickle("dtm_pop.pkl")
data_dtm_hiphop.to_pickle("dtm_hiphop.pkl")

In [ ]:
import pickle

rock_df.to_pickle('data_clean_rock.pkl')
pop_df.to_pickle('data_clean_pop.pkl')
hiphop_df.to_pickle('data_clean_hiphop.pkl')
pickle.dump(cv_rock, open("cv_rock.pkl", "wb"))
pickle.dump(cv_pop, open("cv_pop.pkl", "wb"))
pickle.dump(cv_hiphop, open("cv_hiphop.pkl", "wb"))